In [6]:
import warnings


warnings.filterwarnings("ignore")

import nltk
from nltk import FreqDist
nltk.download('stopwords')

# !pip install pyLDAvis

import pandas as pd
# pd.set_option("display.max_colwidth", 200)
import numpy as np
import json
import re
import gzip
import spacy

import gensim
from gensim import corpora

import pyLDAvis
# import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from gensim.models.coherencemodel import CoherenceModel

from nltk.corpus import stopwords

import nltk

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer

from nltk.stem import WordNetLemmatizer

from tqdm import tqdm

import gensim
from gensim import corpora

import pyLDAvis
# import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tanma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
import json

In [8]:
from gensim.test.utils import datapath

In [9]:
import random

## Load all the required data

In the real world this will be equivalent to querying a table

In [10]:
inter_df = pd.read_csv('./data/interactions.csv')

In [11]:
engaged_df = pd.read_csv("./data/engaged_df.csv")

In [12]:
engaged_df.head()

,book_id,user_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,clean_reviews
0,5805,4cbecbc15af3db041a8e0f594c642bb5,58f2301bd2d4bbfc1b51e4e5fb161cfe,5,"Remember, remember, the fifth of November. Thi...",Wed Jun 13 17:55:53 -0700 2012,Mon Jan 30 05:58:01 -0800 2017,Thu Jun 14 00:00:00 -0700 2012,NaN,7,0,"['remember', 'remember', 'fifth', 'part', 'quo..."
1,5805,49cc59f1c479d698507627b401d47ecf,761a17f52538341a085b629a316204a1,4,Tinha apontado este livro como um dos que tinh...,Sun Jan 05 03:27:10 -0800 2014,Tue Oct 28 15:58:20 -0700 2014,Tue Oct 28 15:58:20 -0700 2014,NaN,1,0,"['tinha', 'visto', 'portman', 'hora', 'livro',..."
2,5805,5f03864c758bfceb6d7d5e93eeb20044,ac54c3ce0c9f660c03881b0668f79c60,5,Review coming soon! www.youtube.com/ReadTomes,Tue Feb 12 11:38:36 -0800 2013,Tue Feb 12 12:00:47 -0800 2013,Tue Feb 12 12:00:47 -0800 2013,Tue Feb 12 00:00:00 -0800 2013,1,0,"['review', 'ing', 'soon', 'readtome']"
3,5805,c309dff1695ed8558b29ea8dcd7479b8,0da0bcc469c2acd15350f9a8f0a74e2b,5,"What better way to celebrate Guy Fawkes Day, t...",Tue May 20 09:52:36 -0700 2014,Thu Mar 16 09:49:11 -0700 2017,Wed Nov 05 14:22:10 -0800 2014,Wed Nov 05 00:00:00 -0800 2014,2,0,"['well', 'way', 'celebrate', 'guy', 'day', 're..."
4,5805,0f6b8c04f811e05c8978bd6b66ce7685,7d670b6c8cac0c086e21ae2f1af6eccb,4,To note - I am writing this review a quarter c...,Tue Jul 03 16:43:26 -0700 2012,Fri Feb 27 14:46:00 -0800 2015,Tue Jan 01 00:00:00 -0800 1991,NaN,10,0,"['note', 'write', 'review', 'quarter', 'centur..."


In [13]:
def to_list(x):
    x = x.strip('][').split(', ')
#     print(x)
    x = [i.strip("'") for i in x]
    return x

In [14]:
engaged_df['clean_reviews'] = engaged_df['clean_reviews'].apply(lambda x: to_list(x))

In [16]:
book_df = pd.read_csv('./data/books_with_topics.csv')

In [17]:
book_df['processed_description'] = book_df['processed_description'].apply(lambda x: to_list(x))

In [18]:
lda_review_model = gensim.models.ldamodel.LdaModel.load("./data/best_review_model")

In [19]:
lda_review_model.print_topics()

[(0,
  '0.016*"con" + 0.011*"" + 0.009*"para" + 0.005*"solo" + 0.004*"star" + 0.004*"son" + 0.004*"mal" + 0.003*"tan" + 0.003*"phoebe" + 0.003*"ser"'),
 (1,
  '0.021*"book" + 0.010*"story" + 0.009*"read" + 0.007*"life" + 0.007*"graphic" + 0.007*"novel" + 0.007*"one" + 0.007*"make" + 0.006*"like" + 0.005*"time"'),
 (2,
  '0.020*"read" + 0.017*"like" + 0.017*"story" + 0.014*"really" + 0.014*"love" + 0.013*"character" + 0.012*"volume" + 0.012*"get" + 0.012*"one" + 0.012*"book"'),
 (3,
  '0.012*"story" + 0.011*"batman" + 0.007*"one" + 0.006*"man" + 0.006*"character" + 0.005*"make" + 0.005*"new" + 0.005*"doe" + 0.005*"book" + 0.005*"take"'),
 (4,
  '0.010*"vader" + 0.009*"kai" + 0.009*"die" + 0.008*"che" + 0.007*"yang" + 0.006*"fairyland" + 0.006*"tintin" + 0.006*"gertrude" + 0.005*"den" + 0.005*"dan"')]

## Get a recommendation for a User

In [20]:
# Let us choose a random user

In [51]:
c = np.random.choice(engaged_df['user_id'].unique())

In [52]:
def generate_recommendations(c, inter_df, engaged_df, book_df, is_debug = True):
    
    #get all the books the user is reviewing
    r_books = engaged_df[engaged_df['user_id'] == c]['book_id'].unique()
    #get all the books the user has interacted with
    r_books = np.concatenate((r_books , inter_df[inter_df['user_id'] == c]['book_id'].unique()))
    
    if is_debug:
        print("These are the books that the user has interacted with:")
        print(book_df[book_df['book_id'].isin(r_books)]['title'].values)
    #Get the distribution of topics
    book_dist = book_df[book_df['book_id'].isin(r_books)]['dom_topic_reviews'].value_counts() 
    book_dist = book_dist/sum(book_dist)
    
    if is_debug:
        
        print(book_dist)
    
    reco = []
    
    count = 0
    
    #Generate a recommnedation using the LDA model
    #If the user has already interacted with the books, just ignore and try to generate a new recommendation
    
    while count< 10:
        idx = random.choices(book_dist.index, weights=book_dist.values)[0]
        id_reco = np.random.choice(book_df[book_df['dom_topic_reviews'] == idx].index)
        if is_debug:
            print(id_reco)
        if book_df.iloc[id_reco]['book_id'] in engaged_df[engaged_df['user_id'] == c]['book_id']:
            pass
        else:
            reco.append((book_df.iloc[id_reco]['title'], idx))
            count = count + 1
    return reco

In [53]:
recos = generate_recommendations(c, inter_df, engaged_df, book_df)

These are the books that the user has interacted with:
["Maus I: A Survivor's Tale: My Father Bleeds History (Maus, #1)"
 'Saga, Vol. 1 (Saga, #1)' 'Preludes & Nocturnes (The Sandman #1)'
 'Persepolis: The Story of a Childhood (Persepolis, #1)'
 'Fables, Vol. 1: Legends in Exile' 'Blankets'
 'Ms. Marvel, Vol. 1: No Normal' 'Saga, Vol. 2 (Saga, #2)'
 'Saga, Vol. 3 (Saga, #3)' 'Nimona'
 "Adulthood Is a Myth (Sarah's Scribbles, #1)" "Anya's Ghost"
 'Saga, Vol. 4 (Saga, #4)' 'Through the Woods'
 'The Stonekeeper (Amulet, #1)'
 'Lumberjanes, Vol. 1: Beware the Kitten Holy' 'Saga, Vol. 5 (Saga, #5)'
 'El Deafo' 'Habibi' 'Paper Girls, Vol. 1 (Paper Girls, #1)'
 'The Wicked + The Divine, Vol. 1: The Faust Act'
 'Rat Queens, Vol. 1: Sass & Sorcery' 'Locke & Key, Vol. 2: Head Games'
 "The Stonekeeper's Curse (Amulet, #2)" 'Saga, Vol. 6 (Saga, #6)'
 'Locke & Key, Vol. 3: Crown of Shadows (Locke & Key, #3)'
 'Locke & Key, Vol. 4: Keys to the Kingdom'
 'Ms. Marvel, Vol. 2: Generation Why'
 "Rat Que

In [54]:
print(recos)

[('Hellboy: Weird Tales, Vol. 1 (Hellboy: Weird Tales, #1)', 2), ('Deadly Class, Vol. 1: Reagan Youth', 2), ('Naruto, Vol. 32:  The Search for Sasuke (Naruto, #32)', 2), ('Angel Sanctuary, Vol. 1', 2), ('Skip Beat!, Vol. 04', 2), ('Vampire Knight, Vol. 3 (Vampire Knight, #3)', 2), ('Sailor Moon, #2 (Sailor Moon, #2)', 2), ('Page by Paige', 1), ('Uzumaki, Vol. 1', 2), ('Mouse Guard: Fall 1152 (Mouse Guard, #1)', 2)]
